In [1]:
import math
import random
import logging
import pandas as pd
from kaggle_environments import make, evaluate

termcolor not installed, skipping dependency
No pygame installed, ignoring import


- 0 - rock
- 1 - paper
- 2 - scissors

In [2]:
logging.basicConfig(filename='game_logs.txt', level=logging.INFO, format='%(message)s', filemode='w')

Последние три агента пользуются функцией get_score для выяснения результата хода.

In [3]:
def get_score(left_move, right_move):
    """
    Определяет результат взаимодействия между двумя жестами.

    Функция принимает два жеста и вычисляет, какой из них выигрывает.
    Если оба жеста равны, возвращает 0 (ничья). Если первый жест (left_move)
    выигрывает, возвращает 1. Если второй жест (right_move) выигрывает,
    возвращает -1.

    Args:
        left_move (int): Жест первого игрока, может быть 0 (камень), 1 (бумага) или 2 (ножницы).
        right_move (int): Жест второго игрока, может быть 0 (камень), 1 (бумага) или 2 (ножницы).

    Returns:
        int: 1, если первый жест выигрывает; -1, если второй жест выигрывает; 0, если ничья.
    """
    
    delta = (
        right_move - left_move
        
        if (left_move + right_move) % 2 == 0
        else left_move - right_move
    )
    return 0 if delta == 0 else math.copysign(1, delta)

Первые три агента будут всегда выбрасывать либо 0, либо 1, либо 2.

In [4]:
def constant_rock(observation, configuration):
    """
    Агент, который всегда выбирает жест "камень".

    Этот агент не меняет свой выбор и постоянно выбрасывает камень 
    в каждом раунде. Это делает его предсказуемым для противника, 
    так как он не адаптируется к действиям соперника.

    Args:
        observation (object): Наблюдение о состоянии текущего раунда, включая:
                              - observation.step (int): текущий номер шага.
                              - observation.reward (int): текущая награда.
        configuration (object): Конфигурация игры, включая:
                                - configuration.signs (int): количество возможных жестов.

    Returns:
        int: Константный жест "камень", представленный целым числом 0.
    """
    
    move = 0 
    
    logging.info(
        f"constant_rock: "
        f"step={observation.step}, "
        f"move={move}, "
        f"reward={observation.reward}"
    )    
    return move

def constant_paper(observation, configuration):
    """
    Агент, который всегда выбирает жест "бумага".

    Этот агент не меняет свой выбор и постоянно выбрасывает бумагу 
    в каждом раунде. Это делает его предсказуемым для противника, 
    так как он не адаптируется к действиям соперника.

    Args:
        observation (object): Наблюдение о состоянии текущего раунда, включая:
                              - observation.step (int): текущий номер шага.
                              - observation.reward (int): текущая награда.
        configuration (object): Конфигурация игры, включая:
                                - configuration.signs (int): количество возможных жестов.

    Returns:
        int: Константный жест "бумага", представленный целым числом 1.
    """
    
    move = 1 
    
    logging.info(
        f"constant_paper: "
        f"step={observation.step}, "
        f"move={move}, "
        f"reward={observation.reward}"
    )    
    return move

def constant_scissors(observation, configuration):
    """
    Агент, который всегда выбирает жест "ножницы".

    Этот агент не меняет свой выбор и постоянно выбрасывает ножницы 
    в каждом раунде. Это делает его предсказуемым для противника, 
    так как он не адаптируется к действиям соперника.

    Args:
        observation (object): Наблюдение о состоянии текущего раунда, включая:
                              - observation.step (int): текущий номер шага.
                              - observation.reward (int): текущая награда.
        configuration (object): Конфигурация игры, включая:
                                - configuration.signs (int): количество возможных жестов.

    Returns:
        int: Константный жест "ножницы", представленный целым числом 2.
    """
    
    move = 2 
    
    logging.info(
        f"constant_scissors: "
        f"step={observation.step}, "
        f"move={move}, "
        f"reward={observation.reward}"
    )    
    return move

Далее будет агент, который всегда выбрасывает рандомные значения.

In [5]:
def random_sign(observation, configuration):
    """
    Агент, который всегда выбирает случайный жест.

    Этот агент не следует никаким стратегиям или шаблонам, а просто
    выбирает жест случайным образом из доступных вариантов. Это может
    сделать его предсказуемым для противника, но также позволяет
    избежать привязки к определенной стратегии.

    Args:
        observation (object): Наблюдение о состоянии текущего раунда, включая:
                              - observation.step (int): текущий номер шага.
                              - observation.reward (int): текущая награда.
        configuration (object): Конфигурация игры, включая:
                                - configuration.signs (int): количество возможных жестов.

    Returns:
        int: Случайный жест агента, выбранный из диапазона [0, configuration.signs - 1].
    """
    
    move = random.randrange(0, configuration.signs)
    
    logging.info(
        f"random_sign: "
        f"step={observation.step}, "
        f"move={move}, "
        f"reward={observation.reward}"
    )    
    return move

Затем два агента с ротацией: sequence всегда чередует [0, 1, 2], а rotation каждые три хода будет случайным образом генерировать свою последовательность.

In [6]:
def create_sequence():
    """
    Создаёт агент, который делает ходы в строгой последовательности.

    Агент выполняет ходы в порядке: камень, ножницы, бумага. После того как
    все жесты будут использованы, последовательность начинается заново.

    Returns:
        function: Функция `sequence`, которая принимает `observation` и
                  `configuration` и возвращает целое число (ход агента)
                  от 0 до `configuration.signs - 1`.
    """
    
    sign = 0
    order = [0, 1, 2] 

    def sequence(observation, configuration):
        """
        Выбирает текущий жест согласно установленной последовательности.

        Args:
            observation (object): Наблюдение о состоянии текущего раунда, включая:
                                  - observation.step (int): текущий номер шага.
                                  - observation.reward (int): текущая награда.
            configuration (object): Конфигурация игры, включая:
                                    - configuration.signs (int): количество возможных жестов.

        Returns:
            int: Жест агента, который следует текущей последовательности.
        """
        
        nonlocal sign
        
        move = order[sign] 
        sign += 1
            
        if sign >= len(order):
            sign = 0 
            
        logging.info(
            f"sequence: "
            f"step={observation.step}, "
            f"move={move}, "
            f"reward={observation.reward}"
        )    
        return move  
    return sequence

def create_rotation():
    """
    Создаёт случайную ротацию ходов и следует ей.

    Агент генерирует случайный порядок ходов (камень, ножницы, бумага) и
    выполняет их по очереди. После завершения ротации порядок обновляется
    и начинается заново.

    Returns:
        function: Функция `rotation`, которая принимает `observation` и
                  `configuration` и возвращает целое число (ход агента)
                  от 0 до `configuration.signs - 1`.
    """
    
    sign = 0
    order = [0, 1, 2]
    random.shuffle(order) 
    
    def rotation(observation, configuration):
        """
        Выбирает текущий жест согласно ротации.

        Args:
            observation (object): Наблюдение о состоянии текущего раунда, включая:
                                  - observation.step (int): текущий номер шага.
                                  - observation.reward (int): текущая награда.
            configuration (object): Конфигурация игры, включая:
                                    - configuration.signs (int): количество возможных жестов.

        Returns:
            int: Жест агента, который следует текущему порядку ротации.
        """
        
        nonlocal sign 
        move = order[sign]  
        sign += 1
        
        if sign >= len(order):
            random.shuffle(order)  
            sign = 0 

        logging.info(
            f"rotation: "
            f"step={observation.step}, "
            f"move={move}, "
            f"reward={observation.reward}"
        )        
        return move
    return rotation

Далее два агента, которые действуют исходя из последнего хода противника: copy_opponent копирует его, respond_to_opponent выбирает противодействующий.

In [7]:
def copy_opponent(observation, configuration):
    """
    Повторяет последний жест противника.

    Агент на первом шаге делает случайный выбор жеста. На всех последующих
    шагах он копирует последний ход противника.

    Returns:
        int: Жест, который будет повторять последний ход противника,
              возвращаемое значение от 0 до `configuration.signs - 1`.
    """
    
    if observation.step > 0:
        move = observation.lastOpponentAction  
    else:
        move = random.randrange(0, configuration.signs)  

    logging.info(
        f"copy_opponent: "
        f"step={observation.step}, "
        f"move={move}, "
        f"reward={observation.reward}"
    )    
    return move

def respond_to_opponent(observation, configuration):
    """
    Выбирает жест, который побеждает предыдущий жест противника.

    Агент на первом шаге делает случайный выбор жеста. На следующих шагах он
    анализирует последний жест противника и выбирает жест, который его контрит.
    
    Returns:
        int: Выбранный жест агента, который будет контрить последний ход
              противника, возвращаемое значение от 0 до `configuration.signs - 1`.
    """
    
    if observation.step == 0:
        move = random.randrange(0, configuration.signs)  
    else:
        opponent_move = observation.lastOpponentAction  
        
        if opponent_move == 0:  
            move = 1  
        elif opponent_move == 1:  
            move = 2  
        elif opponent_move == 2:  
            move = 0  

        logging.info(
            f"respond_to_opponent: "
            f"step={observation.step}, "
            f"move={move}, "
            f"reward={observation.reward}"
        )    
    return move

Далее агент, который накапливает информацию о количестве ходов, которые делает противник, и выбирает контрящий ходу с наибольшим счетчиком.

In [8]:
def create_common_selection():
    """
    Создает агента, который противодействует наиболее частому ходу противника.

    Этот агент отслеживает количество раз, когда противник выбрал каждый из
    возможных жестов (камень, ножницы, бумага), и выбирает жест, который
    контрит наиболее частый ход противника. Если противник часто повторяет
    один и тот же жест, агент будет адаптироваться к этому шаблону.

    Returns:
        function: Функция `common_selection`, которая принимает `observation` и
                  `configuration` и возвращает целое число (ход агента) от 0 до
                  `configuration.signs - 1`.
    """
    
    stats = {0: 0, 1: 0, 2: 0}
    
    def common_selection(observation, configuration):
        """
        Определяет жест агента на основе статистики последних ходов противника.

        Args:
            observation (object): Наблюдение о состоянии текущего раунда, включая:
                                  - observation.step (int): текущий номер шага.
                                  - observation.lastOpponentAction (int): жест, выбранный противником.
            configuration (object): Конфигурация игры, включая:
                                    - configuration.signs (int): количество возможных жестов.

        Returns:
            int: Выбранный жест агента, который будет контрить наиболее частый
                 жест противника.
        """
        
        nonlocal stats
        
        if observation.step > 0:
            last_move = observation.lastOpponentAction
            stats[last_move] += 1  
        else:
            return random.randrange(0, configuration.signs)  

        most_common_move = max(stats, key=stats.get)

        if most_common_move == 0:
            move = 1 
        elif most_common_move == 1:
            move = 2  
        else:
            move = 0 

        logging.info(
            f"common_selection: "
            f"step={observation.step}, "
            f"move={move}, "
            f"reward={observation.reward}, "
            f"stats={stats}"
        )
        return move
    return common_selection

Агент, который проверяет своё текущее состояние reward. Если оно слишком низкое, то он играет по тактике respond_to_opponent, если среднее, то по тактике random_sign, если высокое, то пытается защищаться, создавая ротацию, в которой чаще будут присутствовать только ножницы и иногда разбавляя её камнем или бумагой.

In [9]:
def create_reward_check():
    """
    Создает агента, который принимает решения на основе полученной награды и
    последнего хода противника.

    Этот агент использует три различных стратегии в зависимости от награды:
    - Если награда меньше или равна 25, агент выбирает ход, который контрит
      последний ход противника.
    - Если награда больше 25 и меньше или равна 75, агент выбирает случайный
      ход.
    - Если награда больше 75, агент использует заранее определенный порядок
      ходов, который повторяется после использования трех ходов.

    Returns:
        function: Функция `reward_check`, которая принимает `observation` и
                  `configuration` и возвращает целое число (ход агента) от 0 до
                  `configuration.signs - 1`.
    """
    
    sign = 0
    order = [2, 2, 2, 0, 1]  
    order_def = random.sample(order, 3)
    
    def reward_check(observation, configuration):
        """
        Определяет жест агента на основе текущего шага и награды.

        Args:
            observation (object): Наблюдение о состоянии текущего раунда, включая:
                                  - observation.step (int): текущий номер шага.
                                  - observation.lastOpponentAction (int): жест, выбранный противником.
                                  - observation.reward (int): награда, полученная за предыдущий ход.
            configuration (object): Конфигурация игры, включая:
                                    - configuration.signs (int): количество возможных жестов.

        Returns:
            int: Выбранный жест агента, который будет основан на логике награды и
                 ходах противника.
        """
        
        nonlocal sign, order_def
    
        if observation.step == 0:
            move = random.randrange(0, configuration.signs)
        else:
            if observation.reward <= 25:
                opponent_move = observation.lastOpponentAction
            
                if opponent_move == 0:  
                    move = 1 
                elif opponent_move == 1:  
                    move = 2 
                elif opponent_move == 2:  
                    move = 0 
                         
            elif 25 < observation.reward <= 75:
                move = random.randrange(0, configuration.signs)  
            else:
                move = order_def[sign]
                sign += 1
                
                if sign >= len(order_def):  
                    order_def = random.sample(order, 3)  
                    sign = 0 
                    
        logging.info(
            f"reward_check: "
            f"step={observation.step}, "
            f"move={move}, "
            f"reward={observation.reward} "
        )            
        return move
    return reward_check

Агент, который будет избегать повторений одного жеста дважды подряд.

In [10]:
def create_anti_replay():
    """
    Создает агента, который избегает повторения одного и того же хода дважды подряд.

    Этот агент выбирает случайный ход на первом шаге, а затем на каждом последующем
    выбирает случайный ход, отличный от предыдущего. Это помогает избежать предсказуемости
    для оппонента.

    Returns:
        function: Функция `anti_replay`, которая принимает `observation` и `configuration`
                  и возвращает целое число (ход агента) от 0 до `configuration.signs - 1`.
    """
    
    last_move = None
    order = [0, 1, 2]

    def anti_replay(observation, configuration):
        """
        Выбирает жест, который отличается от последнего выполненного жеста.

        Args:
            observation (object): Наблюдение о состоянии текущего раунда, включая:
                                  - observation.step (int): текущий номер шага.
                                  - observation.lastOpponentAction (int): жест, выбранный противником.
            configuration (object): Конфигурация игры, включая:
                                    - configuration.signs (int): количество возможных жестов.

        Returns:
            int: Выбранный жест агента, который будет отличаться от предыдущего.
        """
        
        nonlocal last_move, order
        
        if observation.step == 0:
            move = random.choice(order)
        else:
            new_order = [i for i in order if i != last_move]
            move = random.choice(new_order)  
            
        last_move = move 
        
        logging.info(
            f"anti_replay: "
            f"step={observation.step}, "
            f"move={move}, "
            f"reward={observation.reward} "
        )     
        return move  
    return anti_replay

Агент, который попробует предсказать ход противника на основе вероятностей.

In [11]:
def create_predictive_agent():
    """
    Создает агента-предсказателя, который выбирает ход на основе вероятности действий противника.

    Агент отслеживает количество каждого хода противника и вычисляет вероятность каждого
    из них. На основе этих вероятностей агент выбирает оптимальный ответ, добавляя
    элемент случайности, чтобы оставаться менее предсказуемым.

    Returns:
        function: Функция `predictive_agent`, которая принимает `observation` и `configuration`
                  и возвращает целое число (ход агента) от 0 до `configuration.signs - 1`.
    """
    action_counts = {0: 0, 1: 0, 2: 0}  
    total_moves = 0  

    def predictive_agent(observation, configuration):
        """
        Выбирает ход на основе вероятностей предыдущих действий противника.

        Args:
            observation (object): Наблюдение о состоянии текущего раунда, включая:
                                  - observation.step (int): текущий номер шага.
                                  - observation.lastOpponentAction (int): жест, выбранный противником.
            configuration (object): Конфигурация игры, включая:
                                    - configuration.signs (int): количество возможных жестов.

        Returns:
            int: Выбранный жест агента, который будет оптимален против наиболее вероятного
                 хода противника.
        """
        nonlocal action_counts, total_moves
        
        if observation.step == 0:
            return random.randrange(0, configuration.signs)  
        
        opponent_move = observation.lastOpponentAction
        action_counts[opponent_move] += 1
        total_moves += 1
        probabilities = {move: count / total_moves for move, count in action_counts.items()}
        best_move = (opponent_move + 1) % configuration.signs  

        if random.random() < 0.8:  
            move = best_move
        else:
            move = random.randrange(0, configuration.signs) 
        
        logging.info(
            f"predictive_agent: "
            f"step={observation.step}, "
            f"move={move}, "
            f"reward={observation.reward}, "
            f"probabilities={probabilities}"
        )             
        return move
    return predictive_agent

- reactionary: стремится контрить последний ход противника, логика такая же как и у respond_to_opponent, но немного иначе реализованная
- counter_reactionary: также контрит последний ход, но если его ход оказался выигрышным, то он повторит свой выигрышный ход
- statistical: продвинутая версия common_selection

In [12]:
def create_reactionary():
    """
    Создает агента, который реагирует на ход противника, выбирая жест, который контрит его предыдущий ход.
    
    Агент делает случайный выбор на первом шаге, а затем на каждом последующем шаге
    выбирает жест, который побеждает последний ход противника, если предыдущий ход агента 
    не выиграл (или ничья). В противном случае агент повторяет предыдущий ход.

    Returns:
        function: Функция `reactionary`, которая принимает `observation` и `configuration`
                  и возвращает целое число (ход агента) от 0 до `configuration.signs - 1`.
    """
    
    move = None

    def reactionary(observation, configuration):
        """
        Определяет ход агента на основе последнего хода противника.

        Args:
            observation (object): Наблюдение о состоянии текущего раунда, включая:
                                  - observation.step (int): текущий номер шага.
                                  - observation.lastOpponentAction (int): жест, выбранный противником.
            configuration (object): Конфигурация игры, включая:
                                    - configuration.signs (int): количество возможных жестов.

        Returns:
            int: Выбранный жест агента, который может быть 0, 1 или 2.
        """
        nonlocal move       
        
        if observation.step == 0:
            move = random.randrange(0, configuration.signs)
        elif get_score(move, observation.lastOpponentAction) <= 1:
            move = (observation.lastOpponentAction + 1) % configuration.signs
        
        logging.info(
            f"reactionary: "
            f"step={observation.step}, "
            f"move={move}, "
            f"reward={observation.reward}, "
        )
        return move
    return reactionary

def create_counter_reactionary():
    """
    Создает агента, который реагирует на ход противника, используя стратегию контр-реакции.

    Агент делает случайный выбор на первом шаге. На последующих шагах:
    - Если предыдущий ход агента побеждает ход противника, агент выбирает жест, который 
      контрит последний ход противника.
    - В противном случае агент выбирает жест, который побеждает ход противника.

    Returns:
        function: Функция `counter_reactionary`, которая принимает `observation` и `configuration`
                  и возвращает целое число (ход агента) от 0 до `configuration.signs - 1`.
    """
    
    move = None
    
    def counter_reactionary(observation, configuration):
        """
        Определяет ход агента на основе последнего хода противника и предыдущего хода агента.

        Args:
            observation (object): Наблюдение о состоянии текущего раунда, включая:
                                  - observation.step (int): текущий номер шага.
                                  - observation.lastOpponentAction (int): жест, выбранный противником.
            configuration (object): Конфигурация игры, включая:
                                    - configuration.signs (int): количество возможных жестов.

        Returns:
            int: Выбранный жест агента, который может быть 0, 1 или 2.
        """
        nonlocal move
        
        if observation.step == 0:
            move = random.randrange(0, configuration.signs)
        elif get_score(move, observation.lastOpponentAction) == 1:
            move = (move + 2) % configuration.signs  
        else:
            move = (observation.lastOpponentAction + 1) % configuration.signs 
        
        logging.info(
            f"counter_reactionary: "
            f"step={observation.step}, "
            f"move={move}, "
            f"reward={observation.reward}, "
        )            
        return move
    return counter_reactionary

def create_statistical():
    """
    Создает агента, который реагирует на ход противника, основываясь на статистике предыдущих ходов.

    Агент собирает историю ходов противника и выбирает жест, который контрит наиболее 
    часто используемый ход противника. На первом шаге он инициализирует историю.

    Returns:
        function: Функция `statistical`, которая принимает `observation` и `configuration`
                  и возвращает целое число (ход агента) от 0 до `configuration.signs - 1`.
    """
    
    action_histogram = {}

    def statistical(observation, configuration):
        """
        Определяет ход агента на основе статистики предыдущих ходов противника.

        Args:
            observation (object): Наблюдение о состоянии текущего раунда, включая:
                                  - observation.step (int): текущий номер шага.
                                  - observation.lastOpponentAction (int): жест, выбранный противником.
            configuration (object): Конфигурация игры, включая:
                                    - configuration.signs (int): количество возможных жестов.

        Returns:
            int: Выбранный жест агента, который может быть 0, 1 или 2.
        """
        nonlocal action_histogram
        
        if observation.step == 0:
            action_histogram = {}  
            return  
        
        action = observation.lastOpponentAction
        
        if action not in action_histogram:
            action_histogram[action] = 0
        action_histogram[action] += 1

        mode_action = None
        mode_action_count = None
        
        for k, v in action_histogram.items():
            if mode_action_count is None or v > mode_action_count:
                mode_action = k
                mode_action_count = v
     
        move = (mode_action + 1) % configuration.signs   
        
        logging.info(
            f"statistical: "
            f"step={observation.step}, "
            f"move={move}, "
            f"reward={observation.reward}, "
        )             
        return move
    return statistical

Итого 15 агентов, запустим их в турнир. Всего 5 раундов, каждый матч по 200 ходов.

In [13]:
env = make("rps")
agents = [
    constant_rock,
    constant_paper,
    constant_scissors,
    create_sequence(), 
    random_sign,
    copy_opponent,
    respond_to_opponent, 
    create_rotation(), 
    create_common_selection(),
    create_reward_check(),
    create_anti_replay(),
    create_predictive_agent(),
    create_reactionary(),
    create_counter_reactionary(),
    create_statistical()
]
results = {agent.__name__: 0 for agent in agents}
match_results = [] 

for i in range(len(agents)):
    for j in range(i + 1, len(agents)):
        agent1, agent2 = agents[i], agents[j]
        
        for match in range(5):
            rewards = evaluate(
                "rps",
                [agent1, agent2],
                configuration={"episodeSteps": 200},
                debug=False
            )[0]

            match_results.append((
                agent1.__name__, 
                agent2.__name__, 
                rewards[0], 
                rewards[1]
            ))

            if rewards[0] > rewards[1]:
                results[agent1.__name__] += 1 
            elif rewards[1] > rewards[0]:
                results[agent2.__name__] += 1 

for agent_name, score in results.items():
    print(f"{agent_name}: {score} побед")

df_rewards = pd.DataFrame(
    match_results, columns=[
        "Agent 1", 
        "Agent 2", 
        "Rewards Agent 1", 
        "Rewards Agent 2"
    ]
)

constant_rock: 5 побед
constant_paper: 5 побед
constant_scissors: 10 побед
sequence: 10 побед
random_sign: 4 побед
copy_opponent: 20 побед
respond_to_opponent: 26 побед
rotation: 30 побед
common_selection: 10 побед
reward_check: 27 побед
anti_replay: 29 побед
predictive_agent: 27 побед
reactionary: 27 побед
counter_reactionary: 41 побед
statistical: 23 побед


Теперь посмотрим на среднее значение reward по агентам.

In [14]:
results_list = []
agents = [
    'constant_rock',
    'constant_paper',
    'constant_scissors',
    'sequence',
    'random_sign',
    'copy_opponent',
    'respond_to_opponent',
    'rotation',
    'common_selection',
    'reward_check',
    'anti_replay',
    'predictive_agent',
    'reactionary',
    'counter_reactionary',
    'statistical'
]

for agent in agents:
    rewards_agent1 = df_rewards[df_rewards['Agent 1'] == agent]['Rewards Agent 1']
    rewards_agent2 = df_rewards[df_rewards['Agent 2'] == agent]['Rewards Agent 2']
    all_rewards = pd.concat([rewards_agent1, rewards_agent2])
    average_reward = all_rewards.mean()
    results_list.append({'Agent': agent, 'Average Reward': average_reward})

final_results = pd.DataFrame(results_list)
final_results.sort_values(by='Average Reward', ascending=False)

,Agent,Average Reward
13,counter_reactionary,64.942857
10,anti_replay,34.200000
11,predictive_agent,28.871429
12,reactionary,28.014286
7,rotation,23.157143
6,respond_to_opponent,22.571429
5,copy_opponent,18.057143
3,sequence,17.614286
14,statistical,15.200000
4,random_sign,0.500000


## Вывод<br>
**Топ агентов по количеству побед:**
- counter_reactionary: 41 
- rotation: 30
- anti_replay: 29
- reactionary: 27
- reward_check: 27
- predictive_agent: 27<br><br>

**Топ агентов по среднему reward:**
- counter_reactionary: 64.94
- anti_replay: 34.2
- predictive_agent:	28.87
- reactionary: 28.01
- rotation: 23.15<br><br>

1. Уверенную победу одерживает `counter_reactionary`, придерживающийся тактики противодействия последнему ходу противника и повторяющий свой выигрышный ход. Именно эта особенность отличает его от `reactionary` и `respond_to_opponent` и даёт ему преимущество по сравнению с ними в наборе с большим количеством агентов так или иначе выставляющих свои ходы случайным образом.<br><br>
2. Агенты `rotation` и `anti_replay`, выставляющие ходы рандомно, но избегающие слишком частых повторений, также продемонстрировали высокую результативность.<br><br>
3. Агенты, опирающиеся на статистику, в данном наборе оказались не слишком эффективны, так как рандомная генерация ходов создаёт примерно одинаковую вероятность каждого хода и затрудняет предсказание следующего хода.